# Рекомендательные системы. Рекомендации через поиск ближайших соседей

> На этом практическом занятии мы с вами сделаем следующее:
- Посмотри как работает FAISS.
- Построим простейший сервис для рекомендаций используя FAISS.

## FAISS

> Необходимо сперва установить faiss библиотеку. Инструкцию можно найти [здесь](https://github.com/facebookresearch/faiss/blob/master/INSTALL.md).
```conda install faiss-cpu -c pytorch # CPU version only```

In [6]:
import numpy as np

In [7]:
dim = 512  # рассмотрим произвольные векторы размерности 512
nb = 10000  # количество векторов в индексе
nq = 1 # количество векторов в выборке для поиска
np.random.seed(58) # DON't CHANGE THIS
vectors = np.random.random((nb, dim)).astype('float32')
query = np.random.random((nq, dim)).astype('float32')

In [ ]:
vectors.shape

In [ ]:
vectors

### IndexFlatL2

Создаем Flat индекс и добавляем векторы без обучения

In [8]:
import faiss # предварительно необходимо установить FAISS см. выше

In [ ]:
index = faiss.IndexFlatL2(dim)
index.add(vectors)
print(index.ntotal)

Проведем поиск по нашим векторам из query:

In [6]:
%%time
topn = 7
D, I = index.search(vectors[:5], topn)

print(I)
print(D)

[[   0 2537    1 2626 8223 2152 1113]
 [   1 5798 2837 3731 4892 2626 6319]
 [   2 6093 5260 5968  983 1208 6507]
 [   3 6267 9639 2341 3631  801 2517]
 [   4  463 3485 4927 7051 5498 8242]]
[[ 0.       72.4341   72.97319  73.29764  73.74105  73.770035 73.9213  ]
 [ 0.       63.8737   67.898994 68.12863  68.509895 68.58003  68.87331 ]
 [ 0.       71.09965  71.71492  72.0022   72.529465 72.5345   72.53931 ]
 [ 0.       70.62649  70.939514 70.94255  71.19     71.44437  72.0357  ]
 [ 0.       71.65051  72.555954 72.78525  73.31938  73.3445   73.62816 ]]
CPU times: user 122 ms, sys: 1.1 ms, total: 123 ms
Wall time: 57 ms


In [ ]:
#Задание 1

In [7]:
%%time
D, I = index.search(query, topn)
print(I)
print(D)

[[3214 8794 9507 6591 8728 3959 5485]]
[[70.102    70.75225  72.44308  72.87785  72.94414  73.420784 73.756744]]
CPU times: user 13.5 ms, sys: 1.2 ms, total: 14.7 ms
Wall time: 6.51 ms


In [ ]:
#Задание 2

### Inverted File Index

Необходмио создать quantiser(IndexFlatL2), индекс (IndexIVFFlat), обучить индекс и добавить вектора в индекс.

In [4]:
#%%time
k = 10 # количество центроидов

quantiser = faiss.IndexFlatL2(dim) 
index = faiss.IndexIVFFlat(quantiser, dim, k)

In [ ]:
index.train(vectors)

In [5]:
index.add(vectors)

RuntimeError: Error in virtual void faiss::IndexIVFFlat::add_core(faiss::Index::idx_t, const float*, const int64_t*, const int64_t*) at /opt/conda/conda-bld/faiss-pkg_1613228596949/work/faiss/IndexIVFFlat.cpp:47: Error: 'is_trained' failed

In [ ]:
D, I = index.search(query, topn)

Необходимо произвести поиск по индексу нашего запроса (query).

## Применим FAISS для рекомендаций в нашей задаче

Построим простейший рекомендательный сервис.

In [9]:
import faiss
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn.decomposition import NMF
from flask import Flask, jsonify, request

# constants
RANDOM_STATE = 57
N_FACTOR = 20 # размерность эмбедингов
N_RESULT = 10 # сколько фильмов рекомендуем

In [10]:
ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")

In [12]:
users = sorted(np.unique(ratings['userId']))
movies = sorted(np.unique(ratings['movieId']))

In [13]:
# for later use
user_id2i = {id: i for i, id in enumerate(users)}
movie_id2i = {id: i for i, id in enumerate(movies)}
movie_i2id = {i: id for i, id in enumerate(movies)}

In [14]:
# make sparse matrix
rating_mat = coo_matrix(
    (ratings['rating'], (ratings['userId'].map(user_id2i), ratings['movieId'].map(movie_id2i)))
)

In [15]:
rating_mat.todense()

matrix([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        ...,
        [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
        [3. , 0. , 0. , ..., 0. , 0. , 0. ],
        [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [16]:
# decompose
model = NMF(n_components=N_FACTOR, init='random', random_state=RANDOM_STATE)
user_mat = model.fit_transform(rating_mat)
movie_mat = model.components_.T

> **NMF** = Non-negative Matrix Factorization. Можно применять метод чередующихся наименьших квадратов (ALS) для неотрицательного матричного разложения. Ключевая идея - искать поочередно то столбцы $p_t$, то столбцы $q_t$ при фиксированных остальных.

In [ ]:
# indexing
# movie_index = faiss.IndexFlatL2(N_FACTOR)

k = 100 # количество центроидов
# необходимо дописать методы
quantiser = faiss.IndexFlatL2(N_FACTOR) 
movie_index = faiss.IndexIVFFlat(quantiser, N_FACTOR, k)
movie_index.train(movie_mat.astype('float32')) 
movie_index.add(movie_mat.astype('float32'))

In [ ]:
# create app
app = Flask(__name__)

In [ ]:
# API endpoint
@app.route('/')
def recom_for_user():
    user_id = request.args.get('user_id', default = 1, type = int)
    user_i = user_id2i[user_id]
    
    user_vec = user_mat[user_i].astype('float32')
    scores, indices = movie_index.search(numpy.array([user_vec]), N_RESULT)
        
    movie_scores = zip(indices[0], scores[0])
    return jsonify(
        movies=[
            {
                "id": int(movie_i2id[i]),
                "score": float(s),
            }
            for i, s in movie_scores
        ],
    )

In [ ]:
app.run(host="0.0.0.0", port=5000)

> Note: use this link in your browser to acccess your server: http://0.0.0.0:5000/?user_id=128